## Installling dependencies

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-la402g3o
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-la402g3o
  Resolved https://github.com/huggingface/transformers.git to commit 66ded238cd04e29ba98485984dd647e7d37d1603
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.6 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7265129 sha256=d9213fce639b36c72baa74ea17a02b844103d6db7b6ee126b5cfbb91de4b3eaa
  Stored in directory: /tmp/pip-ephem-wheel-cache-5qrac72d/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16


In [ ]:
pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.5 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers
!pip install sentencepiece
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 6.5 MB/s eta 0:00:00


## Initialization of model

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import torch

In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(llm_int4_enable_fp32_cpu_offload=True)

tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    load_in_4bit=True,
    quantization_config=quantization_config,
    offload_folder="offload",
    torch_dtype=torch.float16,
    device_map='auto',
)

base_model.tie_weights()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

## Defining prompt and usage

In [ ]:
from langchain import PromptTemplate, LLMChain

template = """Below is an instruction that describes a certain task. Write a long response that fully completes the given request.

### Instruction:
{instruction}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [ ]:
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

question = "Ask me a task about vector addition?"

print(llm_chain.run(question))

 Vector addition is the process of adding two or more vectors together to create a new vector with a summed value in each dimension. For example, if you have two vectors v = [3,4] and w = [-2,-1], then adding them together gives us the resultant vector r = [5,0].
